In [4]:
# Skriptas analizes rezultatų išvedimui iš gitlab testo rezultatų

import pandas as pd
import numpy as np
from pathlib import Path
import json, sys
import neurokit2 as nk

from zive_util_vu import get_label_sums, zive_read_df_data
from zive_util_vu import zive_read_df_rpeaks, get_rec_file_name, zive_read_file_1ch


def misalignments(rec_dir, SubjCode, print_flag=False):

       # I-a dalis: nuskaitome rpeaks iš json failo
    df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    n_rpeaks = atr_sample.shape[0]
    if (print_flag):
        print('    json:',atr_sample[:10])

        # II-a dalis: suformuojame rpeaks su Neurokitu
    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    signal_raw = np.load(file_path)
    _, rpeaks = nk.ecg_peaks(signal_raw, sampling_rate=200, correct_artifacts=False)
    rpeaks_from_signal = rpeaks['ECG_R_Peaks']
    if (print_flag):
        print('neurokit:',rpeaks_from_signal[:10])
  
        # Sulyginimas
    ab = np.setdiff1d(atr_sample, rpeaks_from_signal)
    ba = np.setdiff1d(rpeaks_from_signal,atr_sample)
    return(n_rpeaks, np.size(ab)+np.size(ba))

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Herkulis
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_2022_RUDUO_2'

#  Nuoroda į aplanką su duomenų rinkiniu
# db_path = Path(Duomenu_aplankas, db_folder)

db_path = Path(Duomenu_aplankas, db_folder, "records_selected")

# Failai pūpsnių klasių formavimui
selected_beats = {'N':0, 'S':1, 'V':2}
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3}  

# Čia surašyti failų vardai, kuriuose saugojami testų rezultatai iš gitlab artefact test
# Šie vardai sutampa su testiniais EKG įrašų failų vardais, kurie yra padėti aplanke zive_test_data
file_names = ['1625402.027','1625400.796', '1630757.924', '1631141.764' ]

print("\nLyginami rpeaks iš json, kurie gauti iš frontendo")
print("su rpeaks iš gitlab artefacts:test\n")

for file_name in file_names:
    file_path = Path('gitlab_test_results', file_name)    

    df_data = zive_read_df_data(file_path, "automatic_classification")
    # print(df_data)

    atr_symbol_from_test = df_data['annotation'].to_numpy()
    atr_sample_from_test = df_data['sample'].to_numpy()

    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol_from_test])

    label_sums, total = get_label_sums(test_labels, all_beats)  
    print("file_name: ", file_name, "test_labels: ", list(all_beats.keys()), label_sums, "Total:", total)

# Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su nuskaitytais iš duomenų bazės (atr_sample)
    str = f"Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su nuskaitytais iš duomenų bazės (atr_sample)"
    print(str)
    file_path = Path(db_path, file_name + '.json')    
    # print(file_path)
    with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    df = pd.json_normalize(data, record_path =['rpeaks'])
    
    atr_sample = df['sampleIndex'].to_numpy()
    atr_symbol = df['annotationValue'].to_numpy()

 # Sulyginimas
    ab = np.setdiff1d(atr_sample, atr_sample_from_test)
    ba = np.setdiff1d(atr_sample_from_test,atr_sample)

    print("Nesutampančių rpeaks skaičius:", np.size(ab)+np.size(ba))

# Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su rpeaks, gautais su Neurokit
    
# zive_test_data saugojami testiniai įrašai
    str = f"Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su rpeaks, gautais su Neurokit"
    print(str)
    file_path = Path('zive_test_data', file_name)
    # file_path = Path(rec_dir, file_name)
    signal = zive_read_file_1ch(file_path)      
    _, rpeaks = nk.ecg_peaks(signal, sampling_rate=200, correct_artifacts=False)
    rpeaks_from_signal = rpeaks['ECG_R_Peaks']
    
    # Sulyginimas
    ab = np.setdiff1d(atr_sample_from_test, rpeaks_from_signal)
    ba = np.setdiff1d(rpeaks_from_signal,atr_sample_from_test)

    ns = np.size(ab)+np.size(ba)
    print("Nesutampančių rpeaks skaičius:", ns)


OS in my system :  win32

Lyginami rpeaks iš json, kurie gauti iš frontendo
su rpeaks iš gitlab artefacts:test

file_name:  1625402.027 test_labels:  ['N', 'S', 'V', 'U'] [754   2   5   4] Total: 765
Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su nuskaitytais iš duomenų bazės (atr_sample)
Nesutampančių rpeaks skaičius: 4
Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su rpeaks, gautais su Neurokit
Nesutampančių rpeaks skaičius: 0
file_name:  1625400.796 test_labels:  ['N', 'S', 'V', 'U'] [806   0   9   3] Total: 818
Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su nuskaitytais iš duomenų bazės (atr_sample)
Nesutampančių rpeaks skaičius: 2
Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su rpeaks, gautais su Neurokit
Nesutampančių rpeaks skaičius: 0
file_name:  1630757.924 test_labels:  ['N', 'S', 'V', 'U'] [722   0   4   9] Total: 735
Lyginame rpeaks iš gitlab artefakto test (atr_sample_from_test) su nuskaitytais iš

In [5]:
# Column1	userNr	recordingNr	recId	        file_name	  N	S	 V U	Nml	Sml	Vml	Uml	Nprec	Nrec	Nfsc	Sprec	Srec	Sfsc	Vprec	Vrec	Vfsc	Err%	Noise%

# 10 !	1002	1	60e1db5a93b55b56919e9ec0	1625402.027	748	4	13	0	753	2	5	5	0,99	1,00	0,99	0,50	0,25	0,33	0,80	0,33	0,47	1,70	56,34
# Zive įrašas: 10021 filename: 1625402.027 rpeaks: 765 nesutampančių rpeaks: 4 
# SubjCode: 10021 userNr: 1002 file_name: 1625402.027 userId: 60e1d80f93b55b41529e9eaa recId: 60e1db5a93b55b56919e9ec0 signal_length: 127999
# N:  748 S:  4 V: 13 U:  0  Nml:  753 Sml:  2 Vml:  5 Uml:  5  Nprec: 0.99 Nrec: 1.00 Nfsc: 0.99  Sprec: 0.50 Srec: 0.25 Sfsc: 0.33  Vprec: 0.80 Vrec: 0.33 Vfsc: 0.47  Err%:  1.70  Noise%:  56.34

# 11 	1002	2	60e1db5a93b55b6ab59e9ebe	1625400.796	802	2	14	0	806	0	9	3	0,99	1,00	1,00	0,00	0,00	0,00	0,89	0,57	0,70	1,00	52,93
# Zive įrašas: 10022 filename: 1625400.796 rpeaks: 818 nesutampančių rpeaks: 2
# SubjCode: 10022 userNr: 1002 file_name: 1625400.796 userId: 60e1d80f93b55b41529e9eaa recId: 60e1db5a93b55b6ab59e9ebe signal_length: 127999
# N:  802 S:  2 V: 14 U:  0  Nml:  806 Sml:  0 Vml:  9 Uml:  3  Nprec: 0.99 Nrec: 1.00 Nfsc: 1.00  Sprec: 0.00 Srec: 0.00 Sfsc: 0.00  Vprec: 0.89 Vrec: 0.57 Vfsc: 0.70  Err%:  1.00  Noise%:  52.93


# 48 !	1008	3	613f58593d08d48596cdcce9	1630757.924	723	11	8	0	722	0	9	11	0,99	1,00	1,00	0,00	0,00	0,00	0,56	1,00	0,71	1,50	93,80
# Zive įrašas: 10083 filename: 1630757.924 rpeaks: 742 nesutampančių rpeaks: 7 
# SubjCode: 10083 userNr: 1008 file_name: 1630757.924 userId: 613b1d0c3d08d413ffcdc8f6 recId: 613f58593d08d48596cdcce9 signal_length: 127999
# N:  723 S: 11 V:  8 U:  0  Nml:  722 Sml:  0 Vml:  9 Uml: 11  Nprec: 0.99 Nrec: 1.00 Nfsc: 1.00  Sprec: 0.00 Srec: 0.00 Sfsc: 0.00  Vprec: 0.56 Vrec: 1.00 Vfsc: 0.71  Err%:  1.50  Noise%:  93.80


# 71	1009	1	613b22d53d08d4fe94cdc9ed	1631141.764	845	20	0	0	805	43	15	2	1,00	0,95	0,97	0,12	0,25	0,16	0,00	0,00	0,00	6,50	41,04
# Zive įrašas: 10091 filename: 1631141.764 rpeaks: 865 nesutampančių rpeaks: 0
#  SubjCode: 10091 userNr: 1009 file_name: 1631141.764 userId: 613b1d673d08d4d1f3cdc8f8 recId: 613b22d53d08d4fe94cdc9ed signal_length: 127999
# N:  845 S: 20 V:  0 U:  0  Nml:  805 Sml: 43 Vml: 15 Uml:  2  Nprec: 1.00 Nrec: 0.95 Nfsc: 0.97  Sprec: 0.12 Srec: 0.25 Sfsc: 0.16  Vprec: 0.00 Vrec: 0.00 Vfsc: 0.00  Err%:  6.50  Noise%:  41.04